In [2]:
import torch
from ultra import datasets
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= datasets.Epinions(root= "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data")
test= dataset[2]

# Assuming test.target_edge_index is defined and is a tensor of shape (2, num_edges)
user_ids = test.target_edge_index[0]  # row 0 contains user indices
unique_users, counts = torch.unique(user_ids, return_counts=True)
avg_interactions = counts.float().mean().item()
print("Average test interactions per user:", avg_interactions)



Average test interactions per user: 9.729940414428711


In [5]:
from ultra import datasets

dataset = datasets.Yelp18(root = "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data")
print (f"dataset[0].edge_index.shape{: dataset[0].edge_index.shape}")

dataset[0].edge_index.shapetorch.Size([2, 2194014])


In [ ]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [ ]:
import os
import torch

# Define your input and output directories.
input_dir = "/itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain_full_ckpt"
output_dir = "/itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain"
os.makedirs(output_dir, exist_ok=True)

# Loop over each checkpoint file in the input directory.
for ckpt_file in os.listdir(input_dir):
    # Only process files ending with .pth (or other extensions you expect)
    if not ckpt_file.endswith(".pth"):
        continue

    input_path = os.path.join(input_dir, ckpt_file)
    print(f"Processing {input_path} ...")
    
    # Load the full checkpoint.
    checkpoint = torch.load(input_path, map_location="cpu")
    full_state_dict = checkpoint["model"]

    # Extract the backbone (ultra) state_dict.
    backbone_state_dict = {}
    for key, value in full_state_dict.items():
        if key.startswith("ultra."):
            # Remove the "ultra." prefix.
            new_key = key[len("ultra."):]
            backbone_state_dict[new_key] = value

    # Construct the new checkpoint.
    new_checkpoint = {
        "model": backbone_state_dict,
        "optimizer": checkpoint["optimizer"]
    }
    
    output_path = os.path.join(output_dir, ckpt_file)
    torch.save(new_checkpoint, output_path)
    print(f"Saved backbone checkpoint to {output_path}")


In [5]:
import sqlite3
import pandas as pd


DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db" 
# Connect to the database
conn = sqlite3.connect(DB_FILE)

# Read all entries
df = pd.read_sql("SELECT * FROM experiments", conn)

# Close the connection
conn.close()

# Display the results

print(df.head())  # Use this if running in a terminal


  ckpt         dataset  epochs   bpe  FT   valid_mr  valid_mrr  valid_hits@1  \
0    -    Amazon_Games       8  3520   0  13.169436   0.493378      0.370810   
1    -      Amazon_Men       8  2912   0  24.230614   0.247324      0.160186   
2    -        Epinions       8  8337   0  27.958775   0.276970      0.162142   
3    -  Amazon_Fashion       8  8363   0  23.999451   0.239994      0.158392   
4    -   Amazon_Beauty       8  9184   0  22.984274   0.337647      0.238247   

   valid_hits@3  valid_hits@10  valid_ndcg@10     test_mr  test_mrr  \
0      0.557613       0.728158       0.541948   16.391651  0.411673   
1      0.274220       0.358971       0.285791   24.869064  0.230826   
2      0.305725       0.515802       0.321015  268.101929  0.061357   
3      0.259667       0.326501       0.279042   24.031866  0.231482   
4      0.366086       0.531344       0.371713   26.416218  0.277981   

   test_hits@1  test_hits@3  test_hits@10  test_ndcg@10  valid_ndcg@20  \
0     0.287000    

In [ ]:
%run script/pretrain.py -c config/recommender/pretrain_all.yaml --gpus [0]

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset BookX --epochs 1 --bpe 0 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain/Epinions.pth

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Gowalla --epochs 0 --bpe 20000 --gpus "[0]" --ckpt null

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Epinions --epochs 1 --bpe 10 --gpus "[0]" --ckpt null

In [7]:
import sqlite3

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db"

# Connect to the database
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Manually insert Yelp18 data
yelp18_data = (
    "-", "Yelp18", 8, 17140, 0,  # ckpt, dataset, epochs, bpe, FT
    1006.4, 0.0366413, 0.0136183, 0.0315789, 0.073482, 0.0,  # valid_mr, valid_mrr, valid_hits@1, valid_hits@3, valid_hits@10, valid_ndcg@10
    1992.93, 0.017529, 0.00529698, 0.0132162, 0.034253,  # test_mr, test_mrr, test_hits@1, test_hits@3, test_hits@10
    0.0900301, 0.0676829  # valid_ndcg@20, test_ndcg@20
)

# Manually insert Gowalla data
gowalla_data = (
    "-", "Gowalla", 8, 22265, 0,  # ckpt, dataset, epochs, bpe, FT
    764.808, 0.080052, 0.0424691, 0.0786897, 0.149031, 0.0,  # valid_mr, valid_mrr, valid_hits@1, valid_hits@3, valid_hits@10, valid_ndcg@10
    70747.3, 1.41349e-05, 0, 0, 0,  # test_mr, test_mrr, test_hits@1, test_hits@3, test_hits@10
    0.1098, 0.0  # valid_ndcg@20, test_ndcg@20
)

# Corrected SQL query with double quotes around column names
insert_query = """
INSERT INTO experiments (
    ckpt, dataset, epochs, bpe, FT,
    valid_mr, valid_mrr, "valid_hits@1", "valid_hits@3", "valid_hits@10", "valid_ndcg@10",
    test_mr, test_mrr, "test_hits@1", "test_hits@3", "test_hits@10",
    "valid_ndcg@20", "test_ndcg@20"
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Execute the inserts
cursor.execute(insert_query, yelp18_data)
cursor.execute(insert_query, gowalla_data)

# Commit changes and close connection
conn.commit()
conn.close()

print("Entries for Yelp18 and Gowalla successfully added to the database.")


Entries for Yelp18 and Gowalla successfully added to the database.


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [9]:
import sqlite3
import pandas as pd

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db"

# Connect to the database
conn = sqlite3.connect(DB_FILE)

# Read and filter only the newly inserted datasets
query = 'SELECT * FROM experiments WHERE dataset IN ("Yelp18", "Gowalla")'
df = pd.read_sql(query, conn)

# Close connection
conn.close()
print(df.head())  # Use this if running in a terminal



  ckpt  dataset  epochs    bpe  FT  valid_mr  valid_mrr  valid_hits@1  \
0    -   Yelp18       8  17140   0  1006.400   0.036641      0.013618   
1    -  Gowalla       8  22265   0   764.808   0.080052      0.042469   

   valid_hits@3  valid_hits@10  valid_ndcg@10   test_mr  test_mrr  \
0      0.031579       0.073482            0.0   1992.93  0.017529   
1      0.078690       0.149031            0.0  70747.30  0.000014   

   test_hits@1  test_hits@3  test_hits@10 test_ndcg@10  valid_ndcg@20  \
0     0.005297     0.013216      0.034253         None        0.09003   
1     0.000000     0.000000      0.000000         None        0.10980   

   test_ndcg@20  
0      0.067683  
1      0.000000  


In [4]:
import sqlite3

DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db" 

# Connect to the database
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Delete rows where dataset is 'LastFM' or 'BookX'
cursor.execute("DELETE FROM experiments WHERE dataset IN ('LastFM', 'BookX')")

# Commit changes and close the connection
conn.commit()
conn.close()

print("Rows removed from the database.")


Rows removed from the database.
